In [78]:

from app.log import logging_config
from dotenv import load_dotenv
import logging
from app.util.message import starting_message
from app import Polygon
from app import Sentiment_predict
import pandas as pd
from app.api import Twitter 
from os import environ


## Polygon

In [79]:
##Load and upgrade initial df
polygon_prev = pd.read_csv("polygon_df_01_07_2022.csv")
#Polygon credentials
load_dotenv()
print(environ.get("POLYGON_KEY"))
logging_config.init_logging()
__LOG = logging.getLogger(__name__)
__LOG.info('...... Initialization Completed  ......')
__LOG.info(starting_message())
#Initialize Polygon
polygon = Polygon(environ.get("POLYGON_KEY"))
#Download News
status, polygon_data = polygon.get_news('')
if status:
    polygon_news_df = pd.DataFrame.from_dict(polygon_data['results'])
polygon_news_df.drop(['publisher', 'keywords'], axis = 1, inplace = True)
polygon_news_df['tickers'] = [' '.join(map(str, l)) for l in polygon_news_df['tickers']]
#predict Sentiment
sp = Sentiment_predict()
polygon_news_df['title_finBERT'] = sp.sentiment_finBERT(polygon_news_df, 'title')
polygon_news_df['title_financialBERT'] = sp.sentiment_financialBERT(polygon_news_df, 'title')
polygon_news_df['description_finBERT'] = sp.sentiment_finBERT(polygon_news_df, 'description')
polygon_news_df['description_financialBERT'] = sp.sentiment_financialBERT(polygon_news_df, 'description')
#Upload df.
polygon_news_df = pd.concat([polygon_news_df, polygon_prev], ignore_index = True).drop_duplicates().reset_index(drop = True)
polygon_news_df.to_csv("polygon_df_01_07_2022.csv", index = False, encoding='utf-8')



PWRjSfmBc9xYnPZMQhA9ukDOuOIId90D
in row 699 the text is divided into two parts
in row 944 the text is divided into two parts
in row 699 the text is divided into two parts
in row 944 the text is divided into two parts


In [80]:
polygon_news_df.shape

(1286, 13)

In [81]:
#Polygon plots Data
df_poly = polygon_news_df[['id','published_utc', 'title','description', 'title_finBERT', 'description_finBERT', 'title_financialBERT', 'description_financialBERT', 'article_url']]
df_poly['date'] = df_poly['published_utc'].str[0:10]
df_poly["total_news_day"] = df_poly.groupby('date')['id'].transform(len)
df_poly = df_poly.groupby(['date','title_finBERT']).size().reset_index()
df_poly.rename({0:'count'}, axis = 1, inplace = True)
df_poly['date'] = pd.to_datetime(df_poly['date'], format='%Y-%m-%d')
df_poly

C:\Users\andre\AppData\Local\Temp\ipykernel_20224\1300864731.py:3: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\andre\AppData\Local\Temp\ipykernel_20224\1300864731.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,date,title_finBERT,count
0,2022-06-30,negative,87
1,2022-06-30,neutral,313
2,2022-06-30,positive,80
3,2022-07-01,negative,123
4,2022-07-01,neutral,462
5,2022-07-01,positive,98
6,2022-07-02,negative,27
7,2022-07-02,neutral,82
8,2022-07-02,positive,14


In [82]:
df_poly.dtypes

date             datetime64[ns]
title_finBERT            object
count                     int64
dtype: object

## Tweets

In [83]:
twt = Twitter()
df_twt = twt.get_tweets_df(ticker = 'BTC', query = 'bitcoin OR BTC AND crypto', limit = 100, popular =True)
df_twt['tweet_finBERT'] = sp.sentiment_finBERT(df_twt, 'full_text')
df_twt['tweet_financialBERT'] = sp.sentiment_financialBERT(df_twt, 'full_text')
df_twt.dtypes

id                     object
created_at             object
full_text              object
truncated                bool
retweets                int64
user_name              object
user_verified            bool
user_followers          int64
user_image_url         object
ticker                 object
tweet_finBERT          object
tweet_financialBERT    object
dtype: object

In [84]:
df_twt

,id,created_at,full_text,truncated,retweets,user_name,user_verified,user_followers,user_image_url,ticker,tweet_finBERT,tweet_financialBERT
0,1542876211235397632,Fri Jul 01 14:22:08 +0000 2022,Friday I'm in love 😍\nWhich #Crypto will be my...,False,908,mariapazdelgado,True,161575,http://pbs.twimg.com/profile_images/1512065127...,BTC,neutral,neutral
1,1542883785200619521,Fri Jul 01 14:52:14 +0000 2022,We have another concurrent bear flag shown on ...,False,231,OfficialTravlad,True,316749,http://pbs.twimg.com/profile_images/1542527100...,BTC,neutral,neutral
2,1542960700728832000,Fri Jul 01 19:57:52 +0000 2022,Breaking! Voyager has suspended withdrawals! #...,False,183,TheCryptoLark,True,1023689,http://pbs.twimg.com/profile_images/1482889740...,BTC,negative,neutral
3,1543015091259555840,Fri Jul 01 23:34:00 +0000 2022,Many of the world's richest and most famous in...,False,138,TheCryptoLark,True,1023689,http://pbs.twimg.com/profile_images/1482889740...,BTC,neutral,neutral
4,1543098138612973568,Sat Jul 02 05:04:00 +0000 2022,I am just going to keep stacking #bitcoin and ...,False,141,TheCryptoLark,True,1023689,http://pbs.twimg.com/profile_images/1482889740...,BTC,neutral,neutral
5,1542863478750412800,Fri Jul 01 13:31:33 +0000 2022,El Salvador has doubled down on its #bitcoin c...,False,97,BTCTN,True,2633133,http://pbs.twimg.com/profile_images/1140965421...,BTC,negative,negative
6,1543241259451555842,Sat Jul 02 14:32:42 +0000 2022,"Top #Bitcoin whale accumulates 4816 BTC, spend...",False,99,WhaleCoinTalk,True,132871,http://pbs.twimg.com/profile_images/1495764154...,BTC,neutral,neutral
7,1542948843041570823,Fri Jul 01 19:10:45 +0000 2022,"What do the Terra Luna, Celsius, and Voyager f...",False,71,Bitboy_Crypto,True,858468,http://pbs.twimg.com/profile_images/1503382435...,BTC,neutral,neutral
8,1542749444726263809,Fri Jul 01 05:58:25 +0000 2022,Mechanism by which bitcoin/cryptos CAUSE infla...,False,135,nntaleb,True,871356,http://pbs.twimg.com/profile_images/1200820286...,BTC,neutral,neutral
9,1542742867856195584,Fri Jul 01 05:32:17 +0000 2022,"During the last seven days, #LUNC has risen 96...",False,113,BTCTN,True,2633133,http://pbs.twimg.com/profile_images/1140965421...,BTC,positive,positive


In [85]:
df_twt['full_text'].iloc[35]

'Who wouldn’t be bullish on #Crypto when they wake up and open up their @binance app and see this….. #Bitcoin #BNB #ETH https://t.co/Ig2khFS3Az'

In [86]:
month_number = {'Jan':'01', 'Feb':'02', 'Mar':'03', 'Apr':'04', 'May': '05','Jun':'06', 'Jul':'07', 'Aug':'08', 'Sep':'09', 'Oct':'10', 'Nov':'11', 'Dec':'12'}

df_twt['day'] = df_twt['created_at'].str[8:10]
df_twt['month'] = df_twt['created_at'].str[4:7].map(month_number)
df_twt['year'] = df_twt['created_at'].str[-4:]

df_twt['date'] = df_twt['year'] + '-' + df_twt['month'] + '-' + df_twt['day'] 
#Twitter plots Data
df_twt["total_news_day"] = df_twt.groupby('date')['id'].transform(len)
twt_df = df_twt.groupby(['date', 'tweet_finBERT']).size().reset_index()
twt_df.rename({0:'count'}, axis = 1, inplace = True)
twt_df['date'] = pd.to_datetime(twt_df['date'], format='%Y-%m-%d')
twt_df

,date,tweet_finBERT,count
0,2022-06-23,negative,2
1,2022-06-23,neutral,12
2,2022-06-23,positive,2
3,2022-06-24,neutral,2
4,2022-06-25,neutral,1
5,2022-06-27,negative,1
6,2022-06-27,neutral,1
7,2022-06-28,neutral,1
8,2022-06-29,negative,1
9,2022-06-29,neutral,1


## Dash plots

In [87]:
import plotly.express as px
def sent_pie(df, count, col_name, plot_title):
    fig = px.pie(df, values = count, names = col_name, template='plotly_dark', title = plot_title)
    fig.update_layout(
    legend_traceorder="reversed",
    title={
        'y':0.95,
        'x':0.5,
        'xanchor': 'center',
        'yanchor': 'top',
        'font': dict(
        #family="Courier New, monospace",
        size=25)
        },
    )
    
    return fig
def sent_bar(df, date, count, col_name, plot_title):
    fig = px.bar(df, x=date, y=count, color=col_name, template='plotly_dark', category_orders= {col_name: ["neutral", "negative", "positive"]}, title= plot_title)
    fig.update_layout(
        legend_traceorder="reversed",
        legend_title = '',
        title={
            'y':0.95,
            'x':0.5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
            #family="Courier New, monospace",
            size=25)
            },
        )
    fig.layout.xaxis.dtick = 'D1'
    return fig

In [88]:
sent_pie(twt_df, 'count', 'tweet_finBERT', "Tweets by sentiment").show()

In [89]:
sent_pie(df_poly, 'count', 'title_finBERT', "News by sentiment").show()

In [90]:
sent_bar(twt_df, 'date', 'count', 'tweet_finBERT', "Daily Tweets by Sentiment").show()


In [91]:
sent_bar(df_poly, 'date', 'count', 'title_finBERT', "Daily News by Sentiment").show()